In [1]:
import requests
import time
import tarfile
import boto3
import os
from botocore.exceptions import ClientError
import time
 # Importing variables from variables.py

In [ ]:
import sys
sys.path.append('/path/to/aws_cloud')  # Replace with the absolute path to aws_cloud

from my_dependencies import variables

In [ ]:
def create_s3_bucket(bucket_name):
    try:
        s3 = boto3.client('s3', region_name=variables.AWS_DEFAULT_REGION)
        s3.create_bucket(Bucket=bucket_name, 
                         CreateBucketConfiguration={'LocationConstraint': variables.AWS_DEFAULT_REGION})
        print(f"Bucket {bucket_name} created.")
    except boto3.exceptions.Boto3Error as e:
        print(f"Failed to create bucket: {e}")

def retry_upload(local_path, bucket_name, s3_path, retries=3, delay=2):
    s3 = boto3.client('s3', region_name=variables.AWS_DEFAULT_REGION)
    for attempt in range(retries):
        try:
            s3.upload_file(local_path, bucket_name, s3_path)
            print(f"Uploaded: {s3_path}")
            return
        except ClientError as e:
            print(f"Retry {attempt+1}/{retries} failed for {local_path}. Error: {e}")
            time.sleep(delay)
    print(f"Failed to upload {local_path} after {retries} retries.")

def upload_directory_to_s3(bucket_name, directory_path):
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            try:
                local_path = os.path.join(root, file)
                s3_path = os.path.relpath(local_path, start=directory_path)
                retry_upload(local_path, bucket_name, s3_path)
            except Exception as e:
                print(f"Failed to upload {local_path}. Error: {e}")

In [6]:
import requests
import os

def download_file(url, filename, directory):
    """ Download a file from a given URL and save it in a specified directory """
    # Ensure directory exists
    os.makedirs(directory, exist_ok=True)
    
    # Complete path for the file
    file_path = os.path.join(directory, filename)
    
    response = requests.get(url)
    if response.status_code == 200:
        with open(file_path, 'wb') as file:
            file.write(response.content)
        print(f"Downloaded {filename} to {directory}")
    else:
        print(f"Failed to download {filename}. Status code: {response.status_code}")

# Base URL (adjust as needed)
base_url = 'http://vision.stanford.edu/aditya86/ImageNetDogs/'

# List of files to download
files = ['file_list.mat', 'train_list.mat', 'test_list.mat', 
         'annotations.zip', 'images.zip']

# Directory to save files
save_directory = os.getcwd()  # Current working directory

# Download each file
for file in files:
    download_url = base_url + file
    download_file(download_url, file, save_directory)

# Additional code might be needed for 'images/' and 'annotations/' directories


Failed to download file_list.mat. Status code: 404
Failed to download train_list.mat. Status code: 404
Failed to download test_list.mat. Status code: 404
Failed to download annotations.zip. Status code: 404
Failed to download images.zip. Status code: 404


In [ ]:
# URL of the tar file
url = 'http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar'

# Start the timer
start_time = time.time()

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Save the content of the response as a file
    with open('images.tar', 'wb') as file:
        file.write(response.content)
    print("File downloaded and saved as 'images.tar'")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

# End the timer
end_time = time.time()

# Calculate the duration
duration = end_time - start_time
print(f"Download completed in {duration:.2f} seconds")


In [ ]:
# URL of the Stanford Dogs Dataset
url = 'http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar'

# Download the dataset
try:
    print("Downloading dataset...")
    response = requests.get(url, stream=True)
    response.raise_for_status()

    tar_file_path = 'stanford_dogs_dataset.tar'
    with open(tar_file_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=1024):
            file.write(chunk)
except requests.exceptions.RequestException as e:
    print(f"Error downloading dataset: {e}")
    exit(1)

In [ ]:
# Decompress the tar file
try:
    print("Decompressing the dataset...")
    with tarfile.open(tar_file_path, 'r') as tar:
        tar.extractall(path='stanford_dogs_dataset')
    print('Download and extraction complete!')
except (tarfile.TarError, IOError) as e:
    print(f"Error extracting tar file: {e}")
    exit(1)

In [ ]:

# Create S3 Bucket (if it doesn't exist)
create_s3_bucket(variables.MY_APP_BUCKET_NAME)

# Upload to S3
directory_path = 'stanford_dogs_dataset'  # Local directory with the dataset
print("Uploading to S3...")
upload_directory_to_s3(variables.MY_APP_BUCKET_NAME, directory_path)
print("Upload complete!")